In [10]:
import inspect

import polars as pl 

from mintalib.samples import sample_prices
from mintalib.builder import annotate_parameter
from mintalib.core import calc_sma, calc_atr, calc_macd

from polars.datatypes import Struct, Float64


In [11]:
prices = sample_prices()
prices = pl.from_pandas(prices, include_index=True)
prices


date,open,high,low,close,volume
datetime[ns],f64,f64,f64,f64,i64
1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600
1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800
1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000
1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600
1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600
…,…,…,…,…,…
2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400
2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200
2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800


In [12]:
signature = inspect.signature(calc_sma)
print(signature)
parameters = signature.parameters
parameters

(series, period, *, wrap=False)


mappingproxy({'series': <Parameter "series">,
              'period': <Parameter "period">,
              'wrap': <Parameter "wrap=False">})

In [13]:
from functools import wraps

def get_series_expr(src):
    if isinstance(src, str):
        return pl.col(src)
    elif isinstance(src, pl.Series):
        return src
    else:
        raise ValueError("src must be a string or a Polars Series.")


def series_expression(calc_func):
    def decorator(func):
        name = func.__name__.lower()
        metadata = getattr(calc_func, 'metadata', {})
        output_names = metadata.get('output_names', ())
        output_type = Struct({n: Float64 for n in output_names}) if output_names else Float64
        
        @wraps(func)
        def wrapper(src, *args, **kwargs):
            print(f"Calling {func.__name__} with src={src}, args={args}, kwargs={kwargs}")
            source = get_series_expr(src)

            def batch_func(series):
                output = calc_func(series, *args, **kwargs)
                if output_names:
                    return pl.DataFrame(output._asdict()).to_struct(name)
                return output
            
            expr = source.map_batches(batch_func, return_dtype=output_type)
            expr = expr.struct.unnest() if output_names else expr.alias(name)

            return expr
             
        return wrapper
    return decorator


@series_expression(calc_sma)
def SMA(src, *, period: int): ...


prices.select(SMA('close', period=5))

Calling SMA with src=close, args=(), kwargs={'period': 5}


sma
f64
NaN
NaN
NaN
NaN
0.09206
…
230.256
230.704001
231.326001


In [14]:

@series_expression(calc_macd)
def MACD(src): ...


prices.select(MACD('close'))


Calling MACD with src=close, args=(), kwargs={}


macd,macdsignal,macdhist
f64,f64,f64
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
…,…,…
1.815958,1.313965,0.501993
1.941114,1.439395,0.501719
2.046565,1.560829,0.485736


In [15]:
sig = inspect.signature(calc_atr)
sig


<Signature (prices, period=14, *, wrap=False)>

In [17]:
from functools import wraps

def get_struct_expr(src):
    if src is None:
        return pl.struct("*")
    
    if isinstance(src, str):
        return pl.col(src).struct
    
    if isinstance(src, pl.Expr):
        return pl.struct(src)
        
    raise ValueError(f"Unsupported src type: {type(src)}")


def prices_expression(calc_func):
    def decorator(func):
        name = func.__name__.lower()
        metadata = getattr(calc_func, 'metadata', {})
        output_names = metadata.get('output_names', ())
        output_type = Struct({n: Float64 for n in output_names}) if output_names else Float64
        
        @wraps(func)
        def wrapper(*args, src=None, **kwargs):
            print(f"Calling {func.__name__} with src={src}, args={args}, kwargs={kwargs}")
            source = get_struct_expr(src)

            def batch_func(prices):
                output = calc_func(prices.struct, *args, **kwargs)
                if output_names:
                    return pl.DataFrame(output._asdict()).to_struct(name)
                return output
            
            expr = source.map_batches(batch_func, return_dtype=output_type)
            expr = expr.struct.unnest() if output_names else expr.alias(name)
            
            return expr
        
        return wrapper
    return decorator


@prices_expression(calc_atr)
def ATR(period: int, *, src=None): ...

prices.select(ATR(14))

Calling ATR with src=None, args=(14,), kwargs={}


atr
f64
NaN
NaN
NaN
NaN
NaN
…
4.516121
4.479971
4.39783


In [ ]:


def make_signature(calc_func):
    """creates function signature from core function"""
    sig = inspect.signature(calc_func)
    first_param = None
    
    arg_params = []
    for param in sig.parameters.values():
        if first_param is None:
            first_param = param
            if param.name == "prices":
                continue                
            else:
                param = param.replace(
                    name = "src",
                    annotation='Expr',
                )

        if param.name == "wrap":
            continue

        param = annotate_parameter(param)
        arg_params.append(param)

    if first_param.name == "prices":
        param = inspect.Parameter(
            name="src",
            kind=inspect.Parameter.KEYWORD_ONLY,
            default='*',
            annotation=None
        )
        arg_params.append(param)

    return sig.replace(parameters=arg_params)

sig = make_signature(calc_sma)
print(sig)



In [ ]:

def make_expression(calc_func):
    signature = inspect.signature(calc_func)
    cname = f"core.{calc_func.__name__}"
    fname = calc_func.__name__.removeprefix("calc_").upper()
    signature = make_signature(calc_func)
    buffer = f"@wrap_function({cname})\n"
    buffer += f"def {fname}{signature}: ...\n\n"
    print(buffer)

make_expression(calc_sma)